In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import cv2
import mediapipe as mp
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image as keras_image
from tensorflow.keras.applications.vgg16 import preprocess_input
import joblib
# Load the CSV files
correct_df = pd.read_csv('squat/correct.csv')
too_high_df = pd.read_csv('squat/too_high.csv')
too_low_df = pd.read_csv('squat/too_low.csv')

# Add a label column to each dataframe
correct_df['label'] = 'correct'
too_high_df['label'] = 'too_high'
too_low_df['label'] = 'too_low'

# Combine the dataframes
data_df = pd.concat([correct_df, too_high_df, too_low_df])

# Shuffle the combined dataframe
data_df = data_df.sample(frac=1).reset_index(drop=True)

# Split the data into features and labels
X = data_df.drop(['label', 'image_id'], axis=1)  # Assuming all other columns are features
y = data_df['label']

# Encode the labels to integers
y = y.map({'correct': 0, 'too_high': 1, 'too_low': 2})

# Split the data into a training set and a test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, random_state=42)



# Train the model
model.fit(X_train, y_train)
joblib.dump(model, "new_model.h5")
# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred))

# Use the trained model to make predictions on new data
def predict_new_data(new_data):
    # new_data should be a dataframe with the same structure as the training data
    new_prediction = model.predict(new_data)
    return new_prediction

# Example of using the function
# new_data = pd.DataFrame({'angle': [new_angle]})
# prediction = predict_new_data(new_data)
# print(prediction)


              precision    recall  f1-score   support

           0       0.98      0.98      0.98        66
           1       0.98      0.98      0.98        50
           2       1.00      1.00      1.00        59

    accuracy                           0.99       175
   macro avg       0.99      0.99      0.99       175
weighted avg       0.99      0.99      0.99       175



In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import joblib

# Load the trained RandomForest model
model = joblib.load("new_model.h5")

# Define a dictionary to convert model output to human-readable labels
label_dict = {0: "Correct", 1: "Too High", 2: "Too Low"}

# Initialize mediapipe pose class
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)

# Initialize the video capture object
cap = cv2.VideoCapture(0)  # 0 is typically the built-in webcam

frame_count = 0
frame_skip = 9  # this will process every 10th frame

try:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            continue  # skip empty frames

        # Skip frames to process every 10th frame
        frame_count += 1
        if frame_count % (frame_skip + 1) != 0:
            continue

        # Convert the BGR image to RGB
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Perform pose detection
        results = pose.process(image)

        # Draw pose annotations on the image
        mp_drawing = mp.solutions.drawing_utils
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # Calculate the squat angle if pose landmarks are detected
        if results.pose_landmarks:
            # Extract landmarks
            landmarks = results.pose_landmarks.landmark
            try:
                hip = np.array([landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                                landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y])
                knee = np.array([landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                                 landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y])
                ankle = np.array([landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                                  landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y])
                # Calculate the angle using arccos
                angle = np.arccos(np.dot((hip - knee), (ankle - knee)) /
                                  (np.linalg.norm(hip - knee) * np.linalg.norm(ankle - knee)))
                angle = np.degrees(angle)

                # Prepare the angle data for prediction
                new_data = pd.DataFrame({'angle': [angle]})
                prediction = model.predict(new_data)
                squat_label = label_dict[prediction[0]]
                
                # Print the squat label to the terminal
                print(f'Squat label: {squat_label}')
                
                # Display the prediction on the frame
                cv2.rectangle(image, (5, 5), (300, 60), (0, 0, 0), -1)  # Adds a black rectangle as a background for the text
                cv2.putText(image, f'Squat: {squat_label}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            except Exception as e:
                # Print any exceptions to the terminal
                print(e)
                pass  # If there is any error in the landmark detection, ignore it

        # Convert the RGB image back to BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Display the resulting frame
        cv2.imshow('Squat Posture Evaluation', image)

        # Press 'q' to break out of the loop
        if cv2.waitKey(5) & 0xFF == ord('q'):
            break
finally:
    # When everything is done, release the capture
    cap.release()
    cv2.destroyAllWindows()


In [3]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import os

# Initialize MediaPipe Pose solution
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5)

# Function to calculate angle between three points using arccos
def calculate_angle(a, b, c):
    a = np.array(a)  # First
    b = np.array(b)  # Mid
    c = np.array(c)  # End

    radians = np.arccos(np.dot((b - a), (b - c)) / (np.linalg.norm(b - a) * np.linalg.norm(b - c)))
    angle = np.degrees(radians)

    return angle

# Function to annotate image with keypoints and angles
def annotate_image(image, landmarks, knee_angle, hip_angle):
    annotated_image = image.copy()
    mp_drawing = mp.solutions.drawing_utils
    mp_drawing.draw_landmarks(annotated_image, landmarks, mp_pose.POSE_CONNECTIONS)
    cv2.putText(annotated_image, f'Knee Angle: {int(knee_angle)}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    cv2.putText(annotated_image, f'Hip Angle: {int(hip_angle)}', (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
    return annotated_image

# Directories containing images
directories = ['correct', 'too_high', 'too_low']

# Process each directory
for folder in directories:
    data = []
    for filename in os.listdir(folder):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            image = cv2.imread(f'{folder}/{filename}')
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = pose.process(image_rgb)

            if results.pose_landmarks and results.pose_landmarks.landmark:
                landmarks = results.pose_landmarks.landmark

                # Extract landmarks for the knee angle
                hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                       landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
                ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                         landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
                knee_angle = calculate_angle(hip, knee, ankle)

                # Extract landmarks for the hip angle
                shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                            landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                hip_angle = calculate_angle(shoulder, hip, knee)

                # Annotate image
                annotated_image = annotate_image(image, results.pose_landmarks, knee_angle, hip_angle)
                cv2.imwrite(f'{folder}/annotated_{filename}', annotated_image)
                data.append({'Image': filename, 'Knee Angle': knee_angle, 'Hip Angle': hip_angle})

    # Write data to CSV
    df = pd.DataFrame(data)
    df.to_csv(f'{folder}_angles.csv', index=False)

print("Processing complete.")


Processing complete.


In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import joblib

# Load the CSV files
correct_df = pd.read_csv('./correct_angles.csv')
too_high_df = pd.read_csv('./too_high_angles.csv')
too_low_df = pd.read_csv('./too_low_angles.csv')

# Add a label column to each dataframe
correct_df['label'] = 'correct'
too_high_df['label'] = 'too_high'
too_low_df['label'] = 'too_low'

# Combine the dataframes
data_df = pd.concat([correct_df, too_high_df, too_low_df])

# Shuffle the combined dataframe
data_df = data_df.sample(frac=1).reset_index(drop=True)

# Split the data into features and labels
# Assuming that 'Knee Angle' and 'Hip Angle' are the column names for angles in your CSV
X = data_df[['Knee Angle', 'Hip Angle']]  # Only include knee and hip angles as features
y = data_df['label']

# Encode the labels to integers
y = y.map({'correct': 0, 'too_high': 1, 'too_low': 2})

# Split the data into a training set and a test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Save the trained model
joblib.dump(model, "new_model_withhip.h5")

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred))

# Function to predict new data
def predict_new_data(new_data):
    # Load the trained model
    loaded_model = joblib.load("new_model.h5")
    # Predict using the model
    new_prediction = loaded_model.predict(new_data)
    return new_prediction

# Example of how to use the function
# new_data = pd.DataFrame({'Knee Angle': [new_knee_angle], 'Hip Angle': [new_hip_angle]})
# prediction = predict_new_data(new_data)
# print(prediction)


              precision    recall  f1-score   support

           0       0.98      1.00      0.99        55
           1       1.00      0.98      0.99        51
           2       1.00      1.00      1.00        69

    accuracy                           0.99       175
   macro avg       0.99      0.99      0.99       175
weighted avg       0.99      0.99      0.99       175



In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import joblib

# Load the trained RandomForest model
model = joblib.load("new_model.h5")

# Define a dictionary to convert model output to human-readable labels
label_dict = {0: "Correct", 1: "Too High", 2: "Too Low"}

# Initialize mediapipe pose class
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)

# Initialize the video capture object
cap = cv2.VideoCapture(0)  # 0 is typically the built-in webcam

frame_count = 0
frame_skip = 9  # this will process every 10th frame

try:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            continue  # skip empty frames

        # Skip frames to process every 10th frame
        frame_count += 1
        if frame_count % (frame_skip + 1) != 0:
            continue

        # Convert the BGR image to RGB
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Perform pose detection
        results = pose.process(image)

        # Draw pose annotations on the image
        mp_drawing = mp.solutions.drawing_utils
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # Calculate the squat angles if pose landmarks are detected
        if results.pose_landmarks:
            # Extract landmarks
            landmarks = results.pose_landmarks.landmark
            try:
                # Calculate knee angle
                knee_hip = np.array([landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                                     landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y])
                knee_knee = np.array([landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                                      landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y])
                knee_ankle = np.array([landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                                       landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y])
                knee_angle = np.arccos(np.dot((knee_hip - knee_knee), (knee_ankle - knee_knee)) /
                                       (np.linalg.norm(knee_hip - knee_knee) * np.linalg.norm(knee_ankle - knee_knee)))
                knee_angle = np.degrees(knee_angle)

                # Calculate hip angle
                hip_shoulder = np.array([landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                                         landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y])
                hip_hip = np.array([landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                                    landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y])
                hip_knee = np.array([landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                                     landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y])
                hip_angle = np.arccos(np.dot((hip_shoulder - hip_hip), (hip_knee - hip_hip)) /
                                      (np.linalg.norm(hip_shoulder - hip_hip) * np.linalg.norm(hip_knee - hip_hip)))
                hip_angle = np.degrees(hip_angle)

                # Prepare the data for prediction
                new_data_knee = pd.DataFrame({'Knee Angle': [knee_angle], 'Hip Angle': [0]})  # Dummy value for hip angle
                new_data_hip = pd.DataFrame({'Knee Angle': [0], 'Hip Angle': [hip_angle]})  # Dummy value for knee angle

                # Predict the posture
                knee_prediction = model.predict(new_data_knee)
                hip_prediction = model.predict(new_data_hip)

                # Get labels for knee and hip
                knee_label = label_dict[knee_prediction[0]]
                hip_label = label_dict[hip_prediction[0]]
                
                # Display the predictions on the frame
                                # Display the predictions on the frame
                cv2.rectangle(image, (5, 5), (300, 90), (0, 0, 0), -1)  # Adds a black rectangle as a background for the text
                cv2.putText(image, f'Knee: {knee_label}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                cv2.putText(image, f'Hip: {hip_label}', (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

            except Exception as e:
                # Print any exceptions to the terminal
                print(e)
                pass  # If there is any error in the landmark detection, ignore it

        # Convert the RGB image back to BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Display the resulting frame
        cv2.imshow('Squat Posture Evaluation', image)

        # Press 'q' to break out of the loop
        if cv2.waitKey(5) & 0xFF == ord('q'):
            break
finally:
    # When everything is done, release the capture
    cap.release()
    cv2.destroyAllWindows()

